# IMPORTATION DES LIBRAIRIES

In [ ]:
#Chargement des imports

import spacy
import random
import glob
import string
import argparse
import cv2
import os
import json
import requests
import csv


print("Chargement des imports terminé...")

# FONCTIONS DE PRE TRAITEMENT DES DONNEES D ADRESSES

In [ ]:
def preproc(doc):
    doc2 = ""
    bol = 1
    for i in doc:
        string = ' '.join(i.text.split())
        if string != '':
            if bol:
                bol = 0
            else:
                doc2 += ' '
            doc2 += (string)
    
    return nlp.make_doc(doc2)

In [ ]:
def newLine(doc):
    doc2 = ""
    tabNL = []
    tabNomLieu = ["batiment","residence","immeuble","zone"]
    tabNL.extend(tabNomLieu)
    tabType = ['cp','bp','tsa']
    tabNL.extend(tabType)
    tabComplement = ['bureau','appartement','couloir','escalier',"entree"]
    tabNL.extend(tabComplement)
    tabCedex = ['cedex']
    tabNL.extend(tabCedex)
    
    for i in doc:
        if ( i.text[0].isdigit()):
            if( doc[i.i-1].text.lower() not in tabNL ):
                doc2 += "\n"
        if ( i.text.lower() in tabNL ) :

            doc2 += "\n"
        doc2 += i.text+' '
        
    tabSplit = doc2.split('\n')
    bol = 1 
    
    while ( bol == 1):
        bol = 0
        idx = -1
        
        for index, line in enumerate(tabSplit):
            firstWord = line.split(' ')[0].lower()
            if firstWord in tabComplement:
                idx = index
            if firstWord in tabNomLieu and idx != -1 :
                tmp = tabSplit[idx]
                tabSplit[idx] = line
                tabSplit[index] = tmp
         
                bol = 1

    doc2 = '\n'.join(tabSplit)
        
    return nlp.make_doc(doc2)


# INITIALISATION DU MODELE

In [ ]:
#Création du modèle 

nlp = spacy.blank("fr")
ner = nlp.create_pipe("ner")
nlp.add_pipe(ner)
nlp.add_pipe(preproc, first=True)
nlp.add_pipe(newLine, after='preproc')
#Variable d'entrainement du modèle

TRAINING_DATA_text=[]
TRAINING_DATA_ann=[]
tabLabel=[]
nbFile=0    

print("Modèle initialisé...")

In [ ]:
#On ajoute tout les labels présent dans le modèle

tabLabel = ["Sexe","Prenom","Nom","Role","nomLieu","noRue","nomRue","CodePostal","Ville","Cedex","Type","Pays","complementRue","Identification","Service","etage"]
    
#On ajoute les labels au modèle
for label in tabLabel:
    ner.add_label(label)

print("Tableau de labels initialisé !")

# CHARGEMENT DES DONNES 

In [ ]:
nbLineMax = 2000

In [ ]:
with open('DATA_Adresse/adresse_paris.json') as json_file:
    data = json.load(json_file)
    BldgNbTab=[None]*nbLineMax
    StrtNmTab=[None]*nbLineMax
    
    nb=0
    for d in range(nbLineMax):
        adr = data[d]['fields']['l_adr']
        BldgNbTab[d],StrtNmTab[d] = adr.split(' ', 1)

In [ ]:
villes = []
with open('DATA_Adresse/villes_france.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        villes.append(row[3])
        
random.shuffle(villes)

In [ ]:
import csv
import random
entrepriseCSV = []
with open('DATA_Adresse/entreprises.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=';')
    for row in csv_reader:
        entrepriseCSV.append(row)
        
random.shuffle(entrepriseCSV)

In [ ]:
tabServices = ['Service commercial','Département marketing','Direction financière','Service industriel','Département des ressources humaines']

**Traitement des chaines de caractères**

In [ ]:
#On choisi la casse d'une chaine de caractère au hasard, par exemple si on a : texte = 'Exemple'
def transfName(texte):
    rdm = random.choice([1,2,3,4])

    if rdm == 1 :
        texte = texte.upper() # texte -> EXEMPLE ( majuscule )
    elif rdm == 2 :
        texte = texte.lower() # texte -> exemple ( miniscule )
    elif rdm == 3 :
        texte = texte[0].upper() + texte[1:len(texte)].lower() #texte -> Exemple ( nom propres )
        
    return texte

# FONCTION DE GENERATION D'ADRESSES

In [ ]:
def gen_AdresseProfessionel():
    adresse = ""
    TRAINING_DATA_ann={}
    TRAINING_DATA_ann['entities']= []
    
    #Texte aléatoire ou intègrera l'adresse entre les lignes 0 et 4
    allchar = string.ascii_letters+ string.digits+" "

    randEntreprise = int(random.random()*len(entrepriseCSV))
    entreprise = entrepriseCSV[randEntreprise]
    
    #identification
    
    identification = entreprise[0]
    adresse += transfName(identification)
    TRAINING_DATA_ann.get("entities").append([len(adresse)-len(identification),len(adresse),"Identification"])  
    
    #Service
    
    service = random.choice(tabServices)
    adresse += ' '+transfName(service)
    TRAINING_DATA_ann.get("entities").append([len(adresse)-len(service),len(adresse),"Service"])  

    
    
    
    #Nom du Lieu    
    if(random.random()>0.4):
        #randNomLieu1 = int(random.random()*2+2)
        randNomLieu2 = int(random.random()*4 + 4)
        nomLieu1 = random.choice(["Batiment","Residence","Immeuble","Zone Industrielle"])
        nomLieu2 = ''.join([random.choice( string.ascii_letters + ' ' ) for n in range(randNomLieu2)])
        nomLieu = nomLieu1+" "+nomLieu2
        adresse += "\n"+transfName(nomLieu)
        TRAINING_DATA_ann.get("entities").append([len(adresse)-len(nomLieu),len(adresse),"nomLieu"])  
    
    
    #Nom du Lieu    
    if(random.random()>0.4):
        #randNomLieu1 = int(random.random()*2+2)
        etage2 = int(random.random()*1 + 20)
        etage1 = random.choice(["e","er","eme"])
        etage = str(etage2)+etage1+" étage"
        adresse += "\n"+transfName(etage)
        TRAINING_DATA_ann.get("entities").append([len(adresse)-len(etage),len(adresse),"etage"])  
    
    #Nom du Lieu    
    for i in range( random.choice([1,2]) ):
        if(random.random()>0.4):
            #randNomLieu1 = int(random.random()*2+2)
            if ( random.random() > 0.5 ):
                complementRue2 = int(random.random()*399 + 1)
            else :
                complementRue2 = random.choice( string.ascii_uppercase ) 
            
            complementRue1 = random.choice(["Appartement","Bureau",'couloir','escalier',"entree"])
            complementRue = complementRue1+" "+str(complementRue2)
            adresse += "\n"+transfName(complementRue)
            TRAINING_DATA_ann.get("entities").append([len(adresse)-len(complementRue),len(adresse),"complementRue"])  
    
    try:
        noRue,nomRue = entreprise[6].split(' ',1)
    except ValueError as e:
        noRue,nomRue = entrepriseCSV[0][6].split(' ',1)
        
    if nomRue[0] == ' ':
        nomRue = nomRue[1:]
    
    adresse += "\n{}".format(noRue)
    TRAINING_DATA_ann.get("entities").append([len(adresse)-len('{}'.format(noRue)),len(adresse),"noRue"]) 
    
    if( random.random() > 0.5 ) :
        adresse += ", "
        
    adresse += " "+nomRue
    TRAINING_DATA_ann.get("entities").append([len(adresse)-len(nomRue),len(adresse),"nomRue"])  
    

    if(random.random() > 0.5):
        tabTypeAdresse = ["BP","CP","TSA"]
        typeAdresse = random.choice(tabTypeAdresse)+" {}".format( int(random.random()*9999+1) )
        adresse += "\n"+typeAdresse
        TRAINING_DATA_ann.get("entities").append([len(adresse)-len(typeAdresse),len(adresse),"Type"]) 
    
    randCodePostal = entreprise[7]
    adresse += "\n{}".format(randCodePostal)
    TRAINING_DATA_ann.get("entities").append([len(adresse)-len('{}'.format(randCodePostal)),len(adresse),"CodePostal"]) 
    
    
    ville = entreprise[8]
    adresse += " "+transfName(ville)
    TRAINING_DATA_ann.get("entities").append([len(adresse)-len(ville),len(adresse),"Ville"]) 
    
    if(random.random()>0.5):
        Cedex = "Cedex {}".format( int(random.random()*98+1) )
        adresse += "\n"+Cedex
        TRAINING_DATA_ann.get("entities").append([len(adresse)-len(Cedex),len(adresse),"Cedex"])  

    if(random.random()>0.5):
        Pays = "France"
        adresse += "\n"+Pays
        TRAINING_DATA_ann.get("entities").append([len(adresse)-len(Pays),len(adresse),"Pays"])  
    
    return [adresse,TRAINING_DATA_ann]

print("Fonction créée")

# GENERATION D'UN NOMBRE n D'ADRESSES

In [ ]:
TRAINING_DATA_text=[]
TRAINING_DATA_ann=[]

nAdresse = 200

for i in range(nAdresse):

    [adresse,entite] = gen_AdresseProfessionel()
    
    TRAINING_DATA_text.append(adresse)
    TRAINING_DATA_ann.append(entite)
    
print("Variable d'entrainement initialisée !")

# ENTRAINEMENT DU MODELE

In [ ]:
nlp.begin_training()
#On réentraine le modèle 600 : 80
for itn in range(75):
    losses = {}
    nlp.update(TRAINING_DATA_text, TRAINING_DATA_ann, losses=losses)
    
print("Entrainement fini avec un taux de perte à {}".format(losses))

**Sauvegarde du modèle sur le disque**

In [ ]:
#On enregistre le modèle sur le disque 
with nlp.disable_pipes('preproc', 'newLine'):
    nlp.to_disk('Model/Modele_8Jan')

print("Modèle sauvegardé")

**Sauvegarde des adresses sur le disque**

In [ ]:
for i in range(len(TRAINING_DATA_text)):
    
    f = open("/home/hamza/Notebook/Courrier/TextAnn/adresse"+'{}'.format(i)+".txt","w+")
    f.write(TRAINING_DATA_text[i])
    f.close()


**Sauvegarde des annotations d'adresses sur le disque**

In [ ]:
for i in range(len(TRAINING_DATA_ann)):
    f = open("/home/hamza/Notebook/Courrier/TextAnn/adresse"+'{}'.format(i)+".ann","w+")
    for j in range(len(TRAINING_DATA_ann[i]['entities'])):

        debutEntite = TRAINING_DATA_ann[i]['entities'][j][0]
        finEntite = TRAINING_DATA_ann[i]['entities'][j][1]
        nomEntite = TRAINING_DATA_ann[i]['entities'][j][2]
        
        chaine = "T{} {} {} {} \n".format((j+1),nomEntite,debutEntite,finEntite)
        f.write(chaine)

    f.close()

# JEUX D'ESSAIS

In [ ]:
#On la transforme en texte
texte =  gen_AdresseProfessionel()[0]

#On donne le texte au modèle
doc = nlp(texte)

print("Texte : \n")
print(texte)
print()
print("Formatage du texte : \n")

#On affiche le résultat de la prédiction
print(doc)
print()
print("Prédiction : \n")
for entity in doc.ents:
    print(entity.label_, ' | ', entity.text)

In [ ]:
texte2 = """DURAND SA
SERVICE ACHAT
ZONE INDUSTRIELLE OUEST
25 RUE DES FLEURS
33500 LIBOURNE
"""


#On donne le texte au modèle
doc = nlp(texte2)

print("Texte : \n")
print(texte2)
print()
print("Formatage du texte : \n")

#On affiche le résultat de la prédiction
print(doc)
print()
print("Prédiction : \n")
for entity in doc.ents:
    print(entity.label_, ' | ', entity.text)